### Save Raw Data to Filtered Parquet Files

In [18]:
# load in imports
#!sudo apt-get update
#!sudo apt-get install openjdk-8-jdk
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import StructType, StructField, \
StringType, IntegerType, TimestampType, DateType, FloatType
import time
import pathlib
from pyspark.sql.functions import col, to_date

In [2]:
conf = pyspark.SparkConf().setAll([\
    ('spark.master', 'local[*]'),\
    ('spark.app.name', 'ReduceData')])
spark = SparkSession.builder.config(conf=conf)\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/18 01:52:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# get all csvs to load in
allPaths = [str(x) for x in list(pathlib.Path('../../cephfs/data').glob('*.csv')) if 'glucose_records' in str(x)]

In [4]:
# create date range to iterate over
pathRange = spark.sparkContext.range(10, len(allPaths), 10).collect()
pathRange.append(365) 

In [38]:
glucose_data_schema=StructType([StructField('_c0', IntegerType(),True),
                                StructField('PostDate', TimestampType(),True),
                                StructField('IngestionDate', TimestampType(),True),
                                StructField('PostId', StringType(),True),
                                StructField('PostTime', TimestampType(), True),
                                StructField('PatientId', StringType(), True),
                                StructField('Stream', StringType(), True),
                                StructField('SequenceNumber', StringType(), True),
                                StructField('TransmitterNumber', StringType(), True),
                                StructField('ReceiverNumber', StringType(), True),
                                StructField('RecordedSystemTime', TimestampType(), True),
                                StructField('RecordedDisplayTime', TimestampType(), True),
                                StructField('RecordedDisplayTimeRaw', TimestampType(), True),
                                StructField('TransmitterId', StringType(), True),
                                StructField('TransmitterTime', StringType(), True),
                                StructField('GlucoseSystemTime', TimestampType(), True),
                                StructField('GlucoseDisplayTime', TimestampType(), True),
                                StructField('GlucoseDisplayTimeRaw', StringType(), True),
                                StructField('Value', FloatType(), True),
                                StructField('Status', StringType(), True),
                                StructField('TrendArrow', StringType(), True),
                                StructField('TrendRate', FloatType(), True),
                                StructField('IsBackFilled', StringType(), True),
                                StructField('InternalStatus', StringType(), True),
                                StructField('SessionStartTime', StringType(), True)])

In [47]:
# load in data, select columns to save, create date column, save to parquet format
prevIdx = 0

startTime = time.time()
for idx in pathRange:
    paths = allPaths[prevIdx:idx]

    df = spark.read\
            .format('csv')\
            .option('delimiter', ',')\
            .option("mode", "DROPMALFORMED")\
            .option("header", True)\
            .schema(glucose_data_schema)\
            .load(paths)\
            .select(col("PatientId"), col("Value"), \
                    col("GlucoseDisplayTime"), col("GlucoseDisplayTimeRaw"))
            
    df_toParq = df.withColumn('GlucoseDisplayDate',
                           to_date(col('GlucoseDisplayTime')))
    
    df_toParq.repartition(1).write\
                .mode('overwrite')\
                .parquet('/cephfs/stepped_glucose_data/step0_load/parquet_' + str(prevIdx) + '_to_' + str(idx)) 

    break;
    
    prevIdx = idx    

endTime = time.time()
print(endTime-startTime)

115.51292324066162


In [40]:
df_toParq.take(2)

[Row(PatientId='b+SG81ztOyK5NQEu3e/kUZ6ZyN2Lc23hVHVfzD7jPY0=', Value=0.0, GlucoseDisplayTime=datetime.datetime(2022, 9, 11, 11, 34, 7), GlucoseDisplayTimeRaw='2022-09-11T11:34:07.000-05:00', GlucoseDisplayDate=datetime.date(2022, 9, 11)),
 Row(PatientId='b+SG81ztOyK5NQEu3e/kUZ6ZyN2Lc23hVHVfzD7jPY0=', Value=0.0, GlucoseDisplayTime=datetime.datetime(2022, 9, 11, 9, 44, 6), GlucoseDisplayTimeRaw='2022-09-11T09:44:06.000-05:00', GlucoseDisplayDate=datetime.date(2022, 9, 11))]

In [48]:
from save_to_parquet import *

ModuleNotFoundError: No module named 'save_to_parquet'